In [1]:
import os
import csv
import json
import re
import pandas as pd
import psycopg2
import psycopg2.extras

PM25_PATH = "C:/Users/Krist/Documents/Work/Data Science/projects/Air_Quality/data/raw/pollution/PM2_5_2022.csv"
NO2_PATH  = "C:/Users/Krist/Documents/Work/Data Science/projects/Air_Quality/data/raw/pollution/Nitrogen_Dioxide_2022.csv"

PG_DSN="dbname=airquality user=postgres password=Milian112! host=localhost port=5432"
if not PG_DSN:
    raise RuntimeError("Set PG_DSN in environment (PG_DSN) or hardcode it in the script.")

START_DATE = pd.Timestamp("2022-09-01", tz="UTC")

# ---------- helpers ----------
def pg_conn():
    return psycopg2.connect(PG_DSN)

def norm(s):
    if s is None:
        return None
    s = str(s).strip()
    return s if s != "" else None

def parse_status_units_method(status_text: str):
    """
    Status column often looks like:
      'V ugm-3 (BAM)'
      'V ugm-3 (Ref.eq)'
      'No data,V ugm-3 (BAM)' doesn't happen here; 'No data' is in value col.
    We extract:
      status_code = first token
      units = something like 'ugm-3'
      method = content in parentheses
    """
    if not status_text:
        return None, None, None
    t = status_text.strip()
    # status code = first token (V/P/N/S etc) if present
    m = re.match(r"^([A-Za-z]+)\s+(.*)$", t)
    status_code = None
    rest = t
    if m:
        status_code = m.group(1)
        rest = m.group(2)

    units = None
    method = None
    # method in parentheses
    m2 = re.search(r"\(([^)]+)\)", rest)
    if m2:
        method = m2.group(1).strip()
        rest2 = re.sub(r"\([^)]+\)", "", rest).strip()
    else:
        rest2 = rest.strip()

    # units is whatever remains (e.g. ugm-3)
    if rest2:
        units = rest2

    return status_code, units, method

def read_metadata_rows(path: str):
    """
    Reads the first 10 lines of the file and returns a dict of metadata lists aligned by site index.
    Expected structure based on your file:
      line1: title
      line2: notes
      line3: status legend
      line4: Site Name,,"A",,"B",,...
      line5: Latitude,,...,,
      line6: Longitude,,...,,
      line7: Site Type,,...,,
      line8: Zone,,...,,
      line9: Agglomeration,,...,,
      line10: Local Authority,,...,,
      line11: Date,Time,<value>,Status,<value>,Status,...
    """
    with open(path, "r", encoding="utf-8", newline="") as f:
        reader = csv.reader(f)
        lines = [next(reader) for _ in range(10)]

    def extract_every_other(row):
        # After the first cell label, the pattern is: blank, value, blank, value...
        # In your file it looks like: label,,"Site1",,"Site2",,...
        # csv.reader turns ,, into empty strings.
        # The site entries are at indices 2,4,6,...
        out = []
        for i in range(2, len(row), 2):
            out.append(norm(row[i]))
        return out

    meta = {
        "site_name": extract_every_other(lines[3]),
        "latitude": extract_every_other(lines[4]),
        "longitude": extract_every_other(lines[5]),
        "site_type": extract_every_other(lines[6]),
        "zone": extract_every_other(lines[7]),
        "agglomeration": extract_every_other(lines[8]),
        "local_authority": extract_every_other(lines[9]),
    }

    # Convert lat/lon to float where possible
    def to_float_list(xs):
        out = []
        for x in xs:
            try:
                out.append(float(x) if x is not None else None)
            except ValueError:
                out.append(None)
        return out

    meta["latitude"] = to_float_list(meta["latitude"])
    meta["longitude"] = to_float_list(meta["longitude"])

    n_sites = len(meta["site_name"])
    return meta, n_sites

def build_site_keys(n_sites: int):
    # Stable keys by index (site_0001, site_0002...)
    return [f"site_{i:04d}" for i in range(1, n_sites + 1)]

def load_matrix_file_to_long(path: str, pollutant: str) -> pd.DataFrame:
    """
    Reads the matrix CSV as:
      Date, Time, value1, status1, value2, status2, ...
    Using header=None and skipping first 10 lines.
    Drops the header row at the top of the data block (line 11 in file).
    """
    meta, n_sites = read_metadata_rows(path)
    site_keys = build_site_keys(n_sites)

    # Create deterministic column names: date,time then per site: v/status
    cols = ["date", "time"]
    for sk in site_keys:
        cols += [f"{sk}__value", f"{sk}__status"]

    df = pd.read_csv(
        path,
        skiprows=10,       # skip metadata lines
        header=None,       # file has a header row, but it's messy/repeated
        names=cols,
        engine="python",
    )

    # First row here is actually the "Date,Time,PM2.5...,Status,..." header line -> drop it
    df = df[df["date"] != "Date"].copy()

    # Parse datetime (your file uses ISO date + HH:MM:SS)
    dt_utc = pd.to_datetime(
    df["date"] + " " + df["time"],
    errors="coerce",
    utc=True,
    dayfirst=True
    )

    now_utc = pd.Timestamp.utcnow()
    df["datetime_utc"] = dt_utc
    df = df[df["datetime_utc"].notna()]
    df = df[df["datetime_utc"] <= now_utc + pd.Timedelta(days=1)]
    df = df[df["datetime_utc"] >= START_DATE]

    # Melt into long (site_key, value, status_text)
    out_rows = []
    for sk in site_keys:
        vcol = f"{sk}__value"
        scol = f"{sk}__status"

        tmp = df[["datetime_utc", vcol, scol]].copy()
        tmp.rename(columns={vcol: "value_raw", scol: "status_text"}, inplace=True)
        tmp["site_key"] = sk
        tmp["pollutant"] = pollutant

        # value: numeric where possible; treat "No data" as null
        tmp["value"] = pd.to_numeric(tmp["value_raw"].replace({"No data": None}), errors="coerce")

        out_rows.append(tmp[["pollutant", "site_key", "datetime_utc", "value", "status_text"]])

    long_df = pd.concat(out_rows, ignore_index=True)

    # Extract units/method from status_text
    parsed = long_df["status_text"].fillna("").apply(parse_status_units_method)
    long_df["status_code"] = [p[0] for p in parsed]
    long_df["units"] = [p[1] for p in parsed]
    long_df["method"] = [p[2] for p in parsed]

    # raw json for provenance (optional)
    long_df["raw"] = None

    return long_df, meta, site_keys

def upsert_sites(conn, meta, site_keys):
    rows = []
    for i, sk in enumerate(site_keys):
        rows.append((
            sk,
            meta["site_name"][i],
            meta["latitude"][i],
            meta["longitude"][i],
            meta["site_type"][i],
            meta["zone"][i],
            meta["agglomeration"][i],
            meta["local_authority"][i],
        ))

    sql = """
    INSERT INTO dim_uk_air_sites
      (site_key, site_name, latitude, longitude, site_type, zone, agglomeration, local_authority, geom)
    VALUES %s
    ON CONFLICT (site_key) DO UPDATE SET
      site_name=EXCLUDED.site_name,
      latitude=EXCLUDED.latitude,
      longitude=EXCLUDED.longitude,
      site_type=EXCLUDED.site_type,
      zone=EXCLUDED.zone,
      agglomeration=EXCLUDED.agglomeration,
      local_authority=EXCLUDED.local_authority,
      geom=EXCLUDED.geom,
      loaded_at=now();
    """

    # Build geom in SQL so we don't need shapely
    templ = "(%s,%s,%s,%s,%s,%s,%s,%s, CASE WHEN %s IS NOT NULL AND %s IS NOT NULL THEN ST_SetSRID(ST_MakePoint(%s,%s),4326) ELSE NULL END)"

    # Expand each row to include lon/lat for geom
    expanded = []
    for (sk, name, lat, lon, st, zone, agg, la) in rows:
        expanded.append((sk, name, lat, lon, st, zone, agg, la, lat, lon, lon, lat))

    with conn.cursor() as cur:
        psycopg2.extras.execute_values(cur, sql, expanded, template=templ, page_size=1000)
    conn.commit()

def upsert_hourly(conn, df_long: pd.DataFrame):
    rows = []
    for r in df_long.itertuples(index=False):
        rows.append((
            r.pollutant,
            r.site_key,
            r.datetime_utc.to_pydatetime(),
            None if pd.isna(r.value) else float(r.value),
            norm(r.status_text),
            norm(r.units),
            norm(r.method),
            json.dumps({"status_code": r.status_code})  # keep small; expand later if needed
        ))

    sql = """
    INSERT INTO fact_uk_air_hourly
      (pollutant, site_key, datetime_utc, value, status_text, units, method, raw)
    VALUES %s
    ON CONFLICT (pollutant, site_key, datetime_utc) DO UPDATE SET
      value=EXCLUDED.value,
      status_text=EXCLUDED.status_text,
      units=EXCLUDED.units,
      method=EXCLUDED.method,
      raw=EXCLUDED.raw,
      loaded_at=now();
    """

    with conn.cursor() as cur:
        psycopg2.extras.execute_values(cur, sql, rows, page_size=5000)
    conn.commit()

def rebuild_daily(conn):
    # Full rebuild for simplicity
    with conn.cursor() as cur:
        cur.execute("TRUNCATE TABLE fact_uk_air_daily;")
        cur.execute("""
            INSERT INTO fact_uk_air_daily
              (pollutant, site_key, date_utc, avg_value, n_hours, pct_valid)
            SELECT
              pollutant,
              site_key,
              (datetime_utc AT TIME ZONE 'UTC')::date AS date_utc,
              AVG(value) AS avg_value,
              COUNT(*) AS n_hours,
              100.0 * AVG(CASE WHEN value IS NOT NULL THEN 1 ELSE 0 END) AS pct_valid
            FROM fact_uk_air_hourly
            GROUP BY pollutant, site_key, (datetime_utc AT TIME ZONE 'UTC')::date;
        """)
    conn.commit()

def main():
    print("Parsing PM2.5 file...")
    pm25_long, pm25_meta, pm25_keys = load_matrix_file_to_long(PM25_PATH, "pm25")
    print("PM2.5 long rows:", len(pm25_long))

    print("Parsing NO2 file...")
    no2_long, no2_meta, no2_keys = load_matrix_file_to_long(NO2_PATH, "no2")
    print("NO2 long rows:", len(no2_long))

    # Site keys are index-based; both files should have same site ordering, but we upsert from each anyway
    with pg_conn() as conn:
        upsert_sites(conn, pm25_meta, pm25_keys)
        upsert_sites(conn, no2_meta, no2_keys)

        upsert_hourly(conn, pm25_long)
        upsert_hourly(conn, no2_long)

        rebuild_daily(conn)

    print("Done. Loaded dim_uk_air_sites, fact_uk_air_hourly, fact_uk_air_daily")

if __name__ == "__main__":
    main()

Parsing PM2.5 file...
PM2.5 long rows: 1497415
Parsing NO2 file...
NO2 long rows: 3544875
Done. Loaded dim_uk_air_sites, fact_uk_air_hourly, fact_uk_air_daily
